In [1]:
import sys
sys.path.insert(0, "../..")
import config as cfg
import gc
import os
from tqdm.notebook import tqdm
from helper import check_path, seed_everything
from collections import defaultdict

In [2]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [4]:
EXPERIMENT_FAMILY_NAME = 'keras'
EXPERIMENT_NAME = 'distilbert'
N_RANDOM_SEEDS = 1
N_EPOCHS = 100
BATCH_SIZE = 32

In [5]:
RANDOM_STATE = 77
seed_everything(RANDOM_STATE)

In [6]:
def filter_rare_categories(df: pd.DataFrame) -> pd.DataFrame:
    rare_categories = [12]
    return df.loc[~df[cfg.TARGET].isin(rare_categories)]

In [7]:

train = pd.read_pickle(os.path.join(cfg.PREPROCESSED_DATA_PATH, 'train.pkl'))
test = pd.read_pickle(os.path.join(cfg.PREPROCESSED_DATA_PATH, 'test.pkl'))

train = train.rename(columns={cfg.TEXT_COL: 'text'})
test = test.rename(columns={cfg.TEXT_COL: 'text'})

CLASSES = np.sort(train[cfg.TARGET].unique()).tolist()

# train = filter_rare_categories(train)

In [8]:
# EMB_NAME = 'smaller_LaBSE_15lang'
# train_emb = pd.read_pickle(os.path.join(cfg.DATA_PATH, EMB_NAME, 'train.pkl'))
# test_emb = pd.read_pickle(os.path.join(cfg.DATA_PATH, EMB_NAME, 'test.pkl'))

# train = train.join(train_emb)
# test = test.join(test_emb)

# del train_emb, test_emb; gc.collect()

In [9]:
X_train, y_train = train.drop(cfg.TARGET, axis=1), train[cfg.TARGET]
y_train_onecol = y_train.copy()
y_train = pd.get_dummies(y_train)
assert y_train.shape[1] == len(CLASSES)

In [10]:
pred_proba_oof = pd.DataFrame(data=np.zeros(shape=(len(train), len(CLASSES))), index=train.index, columns=CLASSES)
pred_proba_test = pd.DataFrame(data=np.zeros(shape=(len(test), len(CLASSES))), index=test.index, columns=CLASSES)

In [11]:
from typing import Optional


def df_to_dataset(data: pd.DataFrame, labels: Optional[pd.DataFrame]=None, shuffle: bool=True, batch_size: int=32):
  df = data.copy()
  df = {key: value.values[:, tf.newaxis] for key, value in data.items()}
  
  if labels is None:
    ds = tf.data.Dataset.from_tensor_slices((dict(df),))
  else:
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(data))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds


def get_keras_input(
    data: pd.DataFrame,
    labels: Optional[pd.DataFrame] = None,
    shuffle=False
    ) -> tf.data.Dataset:
    return df_to_dataset(
        data=data,
        labels=labels,
        shuffle=shuffle, 
        batch_size=BATCH_SIZE)
        


In [12]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer("https://tfhub.dev/jeongukjae/distilbert_multi_cased_preprocess/2", name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer("https://tfhub.dev/jeongukjae/distilbert_multi_cased_L-6_H-768_A-12/1", trainable=False, name='encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.3)(net)
    net = tf.keras.layers.Dense(300, activation=None)(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    net = tf.keras.layers.Dense(len(CLASSES), activation=None, name='classifier')(net)
    model = tf.keras.Model(text_input, net)

    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    metrics = tf.keras.metrics.AUC(from_logits=True, name='auc')

    epochs = N_EPOCHS
    n = (len(train) // cfg.N_SPLITS) * (cfg.N_SPLITS-1)
    steps_per_epoch = n // BATCH_SIZE
    num_train_steps = steps_per_epoch * epochs
    num_warmup_steps = int(0.1*num_train_steps)

    init_lr = 3e-4
    optimizer = optimization.create_optimizer(init_lr=init_lr,
                                            num_train_steps=num_train_steps,
                                            num_warmup_steps=num_warmup_steps,
                                            optimizer_type='adamw')

    model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)
    return model
    

In [13]:
leak_test = pd.read_pickle(os.path.join(cfg.DATA_PATH, 'test_leak.pkl'))
leak_mask = leak_test.notnull()

In [14]:
cv = StratifiedKFold(n_splits=cfg.N_SPLITS, random_state=cfg.RANDOM_STATE, shuffle=True)

test_pool = get_keras_input(
        data=test,
        shuffle=False
        )


metrics = defaultdict(list)
fold = 0
for train_idx, val_idx in tqdm(cv.split(X_train, y_train_onecol), total=cfg.N_SPLITS):

    train_pool = get_keras_input(
        data=X_train.iloc[train_idx], 
        labels=y_train.iloc[train_idx],
        shuffle=True
        )

    val_pool = get_keras_input(
        data=X_train.iloc[val_idx], 
        labels=y_train.iloc[val_idx],
        shuffle=False
        )
        
    for random_seed in tqdm(range(N_RANDOM_SEEDS), total=N_RANDOM_SEEDS):

        clf = build_classifier_model()
        
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, min_delta=1e-5, mode='auto', restore_best_weights=True)
        clf.fit(x=train_pool, validation_data=val_pool, epochs=N_EPOCHS, callbacks=[early_stopping])
        
        model_name = f'{EXPERIMENT_NAME}_fold_{fold}_rs_{random_seed}'
        model_path = os.path.join(cfg.MODELS_PATH, EXPERIMENT_FAMILY_NAME, EXPERIMENT_NAME)
        check_path(model_path)
        clf.save(os.path.join(model_path, model_name), include_optimizer=False)
        
        pred_proba_oof_val = tf.nn.softmax(clf.predict(val_pool)).numpy()
        pred_proba_oof_train = tf.nn.softmax(clf.predict(train_pool)).numpy()

        pred_proba_oof.iloc[val_idx, :] += pred_proba_oof_val
        pred_proba_test.iloc[:, :] += tf.nn.softmax(clf.predict(test_pool)).numpy()

        y_val_oof = y_train_onecol.iloc[val_idx]

        val_auc = roc_auc_score(y_val_oof, pred_proba_oof_val , multi_class='ovo', labels=CLASSES)
        metrics['val_auc'].append(val_auc)
        print('val auc', val_auc)

        leak_test_auc_score = roc_auc_score(leak_test.loc[leak_mask], pred_proba_test.loc[leak_mask] / (fold + 1), multi_class='ovo', labels=CLASSES)
        metrics['leak_test_auc_score'].append(leak_test_auc_score)
        print('leak_test_auc_score', leak_test_auc_score)
        
        del clf; gc.collect()
    del train_pool,val_pool; gc.collect() 
        
    fold += 1
pred_proba_oof /= N_RANDOM_SEEDS
pred_proba_test /= (cfg.N_SPLITS * N_RANDOM_SEEDS)

2022-07-22 16:10:48.190777: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-22 16:10:48.223354: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-22 16:10:48.223562: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-22 16:10:48.224281: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

  0%|          | 0/5 [00:00<?, ?it/s]

/home/as/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/100
50/50 [==============================] - 25s 387ms/step - loss: 3.0141 - auc: 0.5423 - val_loss: 2.1890 - val_auc: 0.8503
Epoch 2/100
50/50 [==============================] - 17s 347ms/step - loss: 1.9132 - auc: 0.8568 - val_loss: 1.6569 - val_auc: 0.9014
Epoch 3/100
50/50 [==============================] - 18s 366ms/step - loss: 1.7373 - auc: 0.8935 - val_loss: 1.6033 - val_auc: 0.9131
Epoch 4/100
50/50 [==============================] - 18s 357ms/step - loss: 1.6840 - auc: 0.8991 - val_loss: 1.5445 - val_auc: 0.9170
Epoch 5/100
50/50 [==============================] - 17s 343ms/step - loss: 1.5906 - auc: 0.9092 - val_loss: 1.5060 - val_auc: 0.9203
Epoch 6/100
50/50 [==============================] - 19s 376ms/step - loss: 1.5611 - auc: 0.9117 - val_loss: 1.4597 - val_auc: 0.9250
Epoch 7/100
50/50 [==============================] - 18s 365ms/step - loss: 1.4854 - auc: 0.9175 - val_loss: 1.4183 - val_auc: 0.9281
Epoch 8/100
50/50 [==============================] - 17s 351ms

32/32 [==============================] - 9s 248ms/step
val auc 0.7116577420018738
leak_test_auc_score 0.7702653744498466


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/100
50/50 [==============================] - 24s 393ms/step - loss: 2.7689 - auc: 0.6406 - val_loss: 2.0304 - val_auc: 0.8571
Epoch 2/100
50/50 [==============================] - 18s 356ms/step - loss: 1.8832 - auc: 0.8640 - val_loss: 1.6529 - val_auc: 0.9027
Epoch 3/100
50/50 [==============================] - 18s 355ms/step - loss: 1.7352 - auc: 0.8941 - val_loss: 1.6022 - val_auc: 0.9132
Epoch 4/100
50/50 [==============================] - 18s 356ms/step - loss: 1.6801 - auc: 0.8995 - val_loss: 1.5636 - val_auc: 0.9178
Epoch 5/100
50/50 [==============================] - 18s 358ms/step - loss: 1.6044 - auc: 0.9057 - val_loss: 1.5089 - val_auc: 0.9197
Epoch 6/100
50/50 [==============================] - 18s 357ms/step - loss: 1.5675 - auc: 0.9090 - val_loss: 1.4657 - val_auc: 0.9248
Epoch 7/100
50/50 [==============================] - 18s 355ms/step - loss: 1.5090 - auc: 0.9177 - val_loss: 1.4250 - val_auc: 0.9281
Epoch 8/100
50/50 [==============================] - 18s 354ms

KeyboardInterrupt: 

In [ ]:
oof_auc_score = roc_auc_score(y_train, pred_proba_oof , multi_class='ovo', labels=CLASSES)
print('oof_auc_score', oof_auc_score)

oof_auc_score 0.8803577779703942


In [ ]:
# 0.7620819837965503

In [ ]:
leak_test = pd.read_pickle(os.path.join(cfg.DATA_PATH, 'test_leak.pkl'))
notnull = leak_test.notnull()
leak_test_auc_score = roc_auc_score(leak_test.loc[notnull], pred_proba_test.loc[notnull], multi_class='ovo', labels=CLASSES)
print('leak_test_auc_score', leak_test_auc_score)


leak_test_auc_score 0.855811092897954


In [ ]:
# 0.8399045223216896

In [ ]:
submission = pd.read_csv(cfg.SAMPLE_SUBMIT_PATH).set_index('id')
assert submission.index.equals(pred_proba_test.index)
submission[cfg.TARGET] = pred_proba_test.idxmax(1)

submission_path = os.path.join(cfg.SUBMISSION_PATH, EXPERIMENT_FAMILY_NAME)
check_path(submission_path)
submission.to_csv(os.path.join(submission_path, f'{EXPERIMENT_NAME}.csv'))

pred_proba_oof_path = os.path.join(cfg.OOF_PRED_PATH, EXPERIMENT_FAMILY_NAME)
check_path(pred_proba_oof_path)
pred_proba_oof.to_pickle(os.path.join(pred_proba_oof_path, f'{EXPERIMENT_NAME}.pkl'))

pred_proba_test_path = os.path.join(cfg.TEST_PRED_PATH, EXPERIMENT_FAMILY_NAME)
check_path(pred_proba_test_path)
pred_proba_test.to_pickle(os.path.join(pred_proba_test_path, f'{EXPERIMENT_NAME}.pkl'))